<a href="https://colab.research.google.com/github/MahithaPoduvu/Mahitha_INFO5731_-Spring2023/blob/main/INFO5731_Assignment_Three_fall2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [2]:
import pandas as pd # for dataframes
import numpy as np  # for arrays

import re
import string


from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/MyDrive/twitter_data.csv')
df.head


Mounted at /content/gdrive


<bound method NDFrame.head of     Unnamed: 0                                             tweets
0            0  Since the start of the pandemic, 1,144,461 Ame...
1            1  Masks not required for people staying in outdo...
2            2  Since the start of the pandemic, 1,144,461 Ame...
3            3  We were able to go from original sample to ful...
4            4  Since the start of the pandemic, 1,144,461 Ame...
5            5  @MarahChibwana from Malawi is a medical doctor...
6            6  Day 1123, 26.02.23 at 8AM | India #COVID19 upd...
7            7  #India new #COVID cases crosses 200 after 50da...
8            8  @drayeshaverrall @nzlabour @minhealthnz @chris...
9            9  Since the start of the pandemic, 1,144,461 Ame...
10          10  #Males recovered from mild #COVID19 have basel...>

In [3]:
# for preprocessing data
# removes pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt
#data.head()
#Lowercase all texts
df['tweets'] = df['tweets'].str.lower()
# remove twitter handles (@user)
df['clean_tweets'] = np.vectorize(remove_pattern)(df['tweets'], "@[\w]*")
# remove special characters, numbers and punctuations
df['clean_tweets'] = df['clean_tweets'].str.replace("[^a-zA-Z#]", " ")
# remove short words less tha 3 characters long
#df['clean_tweets'] = df['clean_tweets'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df.head()

<ipython-input-3-24a784d27902>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_tweets'] = df['clean_tweets'].str.replace("[^a-zA-Z#]", " ")


,Unnamed: 0,tweets,clean_tweets
0,0,"since the start of the pandemic, 1,144,461 ame...",since the start of the pandemic ame...
1,1,masks not required for people staying in outdo...,masks not required for people staying in outdo...
2,2,"since the start of the pandemic, 1,144,461 ame...",since the start of the pandemic ame...
3,3,we were able to go from original sample to ful...,we were able to go from original sample to ful...
4,4,"since the start of the pandemic, 1,144,461 ame...",since the start of the pandemic ame...


In [68]:

#N-grams
import collections
import nltk
from nltk.util import ngrams
from textblob import TextBlob
nltk.download('punkt')
a = (df['clean_tweets'].str.count('people'))
print(a.sum())
trigrams = []
#TextBlob(df['clean_tweets'][0]).ngrams(3)
for tweet in df['clean_tweets'] :
  trigrams += ngrams(tweet.split(), 3)
pd.Series(trigrams).value_counts(normalize=True)

bigrams = []
for tweet in df['clean_tweets'] :
  bigrams += ngrams(tweet.split(), 2)
pd.Series(trigrams).value_counts(normalize=True)



for element in bigrams :
  print(element)
  print(bigrams.count(element)/(df['clean_tweets'].str.count(element[1]).sum()))






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


5
('since', 'the')
0.2857142857142857
('the', 'start')
0.6666666666666666
('start', 'of')
0.4444444444444444
('of', 'the')
0.2857142857142857
('the', 'pandemic')
0.6666666666666666
('pandemic', 'americans')
1.0
('americans', 'have')
0.8
('have', 'died')
1.0
('died', 'from')
0.25
('from', '#covid')
0.3333333333333333
('#covid', 'of')
0.4444444444444444
('of', 'all')
0.8
('all', 'deaths')
0.6666666666666666
('deaths', 'worldwide')
1.0
('worldwide', 'that')
1.0
('that', 'is')
0.3333333333333333
('is', 'about')
1.0
('about', 'the')
0.2857142857142857
('the', 'same')
1.0
('same', 'as')
0.16666666666666666
('as', 'alaska')
1.0
('alaska', 'airlines')
1.0
('airlines', 'flight')
1.0
('flight', 's')
0.0070921985815602835
('s', 'which')
0.25
('which', 'killed')
1.0
('killed', 'people')
0.8
('people', 'in')
0.12903225806451613
('in', 'https')
0.36363636363636365
('https', 't')
0.07482993197278912
('t', 'co')
0.39285714285714285
('co', 'bdibip')
1.0
('bdibip', 'wt')
1.0
('masks', 'not')
1.0
('not',

# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
# Write your code here







# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Reference: https://jaketae.github.io/study/word2vec/

In [ ]:
# Write your code here







# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file



# Link: 



